## Парсинг данных с сайта на Python

### Определения

Парсинг - это процесс сбора данных с последующей их обработкой и анализом.

Программа, которая занимается парсингом, называют - парсер.


### Условие задачи

С сайта ( https://habr.com/ru/search/ ) необходимо построить исходный набор данных (.csv или .xml). Набор данных должен включать __названия, описание, рейтинг и сферу деятельности компаний, дату публикации, а также текст статей из Интернет-ресурсов__. Подготовленный набор данных должен содержать сведения о всех номинантах конкурса. Разработанный парсер должен извлекать гиперссылки из начальной страницы с последующим обходом всех страниц по полученным ссылкам и извлечением их содержимого. Можно дополнить набор какими-либо другими данными, если они могут быть полезны для дальнейшего исследования.


### Этапы парсинга

1. Поиск данных
2. Получение информации
3. Сохранение данных

### Подключение библиотек

In [1]:
from bs4 import BeautifulSoup as bs

Beautiful Soup - это библиотека Python для извлечения данных из HTML и XML файлов. 

In [2]:
import requests

Библиотека requests является стандартным инструментом для составления HTTP-запросов в Python.

In [3]:
import pandas as pd

### Получение информаций

In [4]:
# GET - запрос
url = 'https://habr.com/ru/all/' # страница со всеми статьями 
page = requests.get(url)

Метод __requests.get(url)__ из библиотеки requests в Python выполняет HTTP-запрос типа GET по указанному URL. Этот запрос используется для получения данных с веб-страницы или API, в нашем случае из страницы habr.

In [5]:
page.status_code

200

Если вызвать __page.status_code__, то получим статус состояния HTTP. например, 200 — успешно, 404 — страница не найдена, 500 — ошибка сервера 


In [6]:
soup = bs(page.text, 'html.parser')

__bs__ — это сокращение от BeautifulSoup, основного класса библиотеки Beautiful Soup.

__bs(page.text, 'html.parser')__ создаёт объект BeautifulSoup, который парсит HTML-код из page.text с использованием указанного парсера.

__'html.parser'__ — это встроенный парсер Python, который не требует установки дополнительных библиотек.а.

In [7]:
page.text

'<!DOCTYPE html>\n<html lang="ru">\n\n  <head>\n    <title>Все статьи подряд &#x2F; Хабр</title>\n<link rel="image_src" href="/img/habr_ru.png" data-hid="2a79c45">\n<link href="https://habr.com/ru/articles/" rel="canonical" data-hid="e3fa780">\n<link href="https://habr.com/ru/articles/" hreflang="ru" rel="alternate" data-hid="7d51b8a">\n<link href="https://habr.com/en/articles/" hreflang="en" rel="alternate" data-hid="7d51b8a">\n<meta itemprop="image" content="/img/habr_ru.png">\n<meta property="og:image" content="/img/habr_ru.png">\n<meta property="og:image:width" content="1200">\n<meta property="og:image:height" content="630">\n<meta property="aiturec:image" content="/img/habr_ru.png">\n<meta name="twitter:image" content="/img/habr_ru.png">\n<meta property="vk:image" content="/img/habr_ru.png?format=vk">\n<meta property="fb:app_id" content="444736788986613">\n<meta property="fb:pages" content="472597926099084">\n<meta name="twitter:card" content="summary_large_image">\n<meta name="tw

In [8]:
soup

<!DOCTYPE html>

<html lang="ru">
<head>
<title>Все статьи подряд / Хабр</title>
<link data-hid="2a79c45" href="/img/habr_ru.png" rel="image_src"/>
<link data-hid="e3fa780" href="https://habr.com/ru/articles/" rel="canonical"/>
<link data-hid="7d51b8a" href="https://habr.com/ru/articles/" hreflang="ru" rel="alternate"/>
<link data-hid="7d51b8a" href="https://habr.com/en/articles/" hreflang="en" rel="alternate"/>
<meta content="/img/habr_ru.png" itemprop="image"/>
<meta content="/img/habr_ru.png" property="og:image"/>
<meta content="1200" property="og:image:width"/>
<meta content="630" property="og:image:height"/>
<meta content="/img/habr_ru.png" property="aiturec:image"/>
<meta content="/img/habr_ru.png" name="twitter:image"/>
<meta content="/img/habr_ru.png?format=vk" property="vk:image"/>
<meta content="444736788986613" property="fb:app_id"/>
<meta content="472597926099084" property="fb:pages"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="@habr_com" name=

Создадим словарь, в который будем записывать данные по заданию: название статьи, описание, рейтинг и сферу деятельности компаний, дату публикации, а также текст статьи из Интернет-ресурса

In [9]:
result_list = {'title': [], 'namecompany': [], 'description': [], 'rating': [], 'field': [], 'date': [], 'textpub': []}

### Алгоритм

Суть алгоритма заключается в переборе страниц, и переходе на "вложенные" страницы, то есть у нас есть основная страница https://habr.com/ru/all/, мы перебираем несколько стараниц с page1 до page10. На каждой странице есть статьи, записываем их в список, чтобы перейти по ним используем  _-i.a.get('href')-_  то есть берём значение из href этого заголовка. Далее находим классы элементов которые нам нужны, и записываем их в результат.

In [10]:
pagenum = 1
for i in range(2):
    url = 'https://habr.com/ru/articles/page' + str(pagenum) + '/' # переход на ссылуку с определённым номером сраницы
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    titles = soup.find_all('h2', class_='tm-title tm-title_h2')# получаем заголовки всех статей на этой странице
    
    for i in titles: 
        # переход на страницу статьи
        url = 'https://habr.com' + str(i.a.get('href')) 
        page = requests.get(url)
        soup = bs(page.text, 'html.parser')
        
        name_company = soup.find('a', class_='tm-company-snippet__title')# получаем название компаний
        desc_company = soup.find('div', class_='tm-company-snippet__description')# получаем описание компаний
        
        if (name_company is not None): #если на странице присутсвует компания
        
            result_list['title'].append(i.text) # записываем название статьи
            result_list['namecompany'].append(name_company.text) # записываем название компании
            result_list['description'].append(desc_company.text) # записываем описание компании
            
            datepub = soup.find('span', class_='tm-article-datetime-published') # находим дату публикаций
            result_list['date'].append(datepub.time['datetime'][0: 10]) # записываем дату публикаций
            
            # текст статьи
            try:
                textpub = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2').get_text()
                textpub = textpub.replace('\n', ' ').replace('\t', ' ').replace('\xa0', ' ').replace('\u200e', ' ').replace('\r', ' ')
            except:
                textpub = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1').get_text()
                textpub = textpub.replace('\n', ' ').replace('\t', ' ').replace('\xa0', ' ').replace('\u200e', ' ').replace('\r', ' ')
            result_list['textpub'].append(textpub)
            
            # переход на страницу компании
            url = 'https://habr.com' + str(name_company.get('href'))
            page = requests.get(url)
            soup = bs(page.text, 'html.parser')
            
            #записываем рейтинг               'tm-votes-lever__score-counter tm-votes-lever__score-counter tm-votes-lever__score-counter_rating'
            rating = soup.find('div', class_='tm-counter-container__header')

            if 'Оценка компании на Хабр Карьере' in rating:
                rating = rating.split('Оценка компании на Хабр Карьере')[1]
            
            if(rating is None):
                result_list['rating'].append('0')
            else:
                result_list['rating'].append((rating.text).strip())
               
             #записываем отрасли компаний
            fieldtext = ""
            fields = soup.find_all('a', 'tm-company-profile__categories-text')
            for field in fields:
                fieldtext = fieldtext + ((field.text).strip()) + ", "
            if (fields is None):
                result_list['field'].append(None)
            else:
                result_list['field'].append(fieldtext[0:-2])
            
    pagenum += 1

In [11]:
result_list

{'title': ['Парсер для подростков с помощью pyparsing',
  'Синтетические данные в 2025: волшебная таблетка для нейросетей или темная лошадка?',
  'От бесплатного курса к оплачиваемой стажировке: интервью с выпускником школы Metaclass',
  'Детективная история с дебагом проблем C-уровня в Python',
  'Топ самых интересных CVE за февраль 2025 года',
  'BotHub, GPTunnel, Chad AI — считаем где ChatGPT дешевле в России',
  'Где в России айтишнику жить хорошо: как работают и зарабатывают ИТ-специалисты в Приволжье',
  'Архитектура приложений: виды, проектирование, разработка',
  'Что такое тёмная паутина и опасна ли она?',
  'Микросервисы в МТС: когда масштаб имеет значение',
  'Техподдержка: стереотипы и правда',
  'Мониторинг контейнеров с помощью cAdvisor',
  'Гибридный Claude 3.7 Sonnet, ребрендинг OpenAI: главные события февраля в сфере ИИ',
  'Сохранение ускользающей истории: как энтузиасты ищут неизвестные мобильные игры',
  'Рекламный сбор: кто и как должен платить налог'],
 'namecompa

In [12]:
print("Количество нулевых значений в: ")
for i in result_list:
    print( i + " - " + str(result_list[i].count(None)))

Количество нулевых значений в: 
title - 0
namecompany - 0
description - 0
rating - 0
field - 0
date - 0
textpub - 0


### Сохранение данных

In [13]:
#file_name = 'habr.csv'
df = pd.DataFrame(data=result_list)
#df.to_csv(file_name)

In [14]:
df.head(10)

,title,namecompany,description,rating,field,date,textpub
0,Парсер для подростков с помощью pyparsing,OTUS,Цифровые навыки от ведущих экспертов,701.22,"Консалтинг и поддержка, Рекрутинг и HR, Произв...",2025-03-04,"Привет, Хабр!Если ты подросток и начинаешь сво..."
1,Синтетические данные в 2025: волшебная таблетк...,Skillfactory,Учим работать в IT на курсах и в магистратурах,70.09,Консалтинг и поддержка,2025-03-04,"Александр РыжковМентор Skillfactory, руководит..."
2,От бесплатного курса к оплачиваемой стажировке...,KTS,Создаем цифровые продукты для бизнеса,203.05,"Веб-разработка, Программное обеспечение, Веб-с...",2025-03-04,Привет!Уже больше шести лет компания KTS прово...
3,Детективная история с дебагом проблем C-уровня...,AvitoTech,У нас живут ваши объявления,384.24,"Веб-разработка, Электронная коммерция, Веб-сер...",2025-03-04,"Привет! Меня зовут Иван Кравцов, я инженер в к..."
4,Топ самых интересных CVE за февраль 2025 года,T.Hunter,Компания,92.43,"Программное обеспечение, Консалтинг и поддержк...",2025-03-04,"⚠ Внимание ⚠Вся информация, представленная ниж..."
5,"BotHub, GPTunnel, Chad AI — считаем где ChatGP...",BotHub,"Агрегатор нейросетей: ChatGPT, Claude, Midjourney",433.66,"Веб-разработка, Программное обеспечение, Веб-с...",2025-03-04,Сейчас на фоне стремительной популяризации ней...
6,Где в России айтишнику жить хорошо: как работа...,Нетология,Источник знаний для роста в профессии,97.91,"Веб-разработка, Производство мультимедиа-конте...",2025-03-04,Продолжаем рубрику «Где нас нет» о жизни ИТ-сп...
7,"Архитектура приложений: виды, проектирование, ...",Слёрм,"Учебный центр для тех, кто работает в IT",148.18,"Веб-разработка, Связь и телекоммуникации",2025-03-04,Как построить архитектуру приложений?Когда реч...
8,Что такое тёмная паутина и опасна ли она?,Сбер,"Технологии, меняющие мир",410.76,"Программное обеспечение, Мобильные технологии,...",2025-03-04,Хотя большинство людей выходят в интернет чере...
9,Микросервисы в МТС: когда масштаб имеет значение,МТС,Про жизнь и развитие в IT,2149.68,"Связь и телекоммуникации, Мобильные технологии...",2025-03-04,"Привет, Хабр!Меня зовут Михаил Федяев, я работ..."


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        15 non-null     object
 1   namecompany  15 non-null     object
 2   description  15 non-null     object
 3   rating       15 non-null     object
 4   field        15 non-null     object
 5   date         15 non-null     object
 6   textpub      15 non-null     object
dtypes: object(7)
memory usage: 972.0+ bytes
